# GETTING FAMILIAR WITH THE DATA 

# Table of contents
1. [Forex Factory](#forex)
    - [Initial exploration](#forex_explore)
    - [Data curation & feature extraction](#next_forex)
2. [Forexite](#forexite)
    - [Initial exploration](#forexite_explore)
    - [Data curation & feature extraction](#next_forexite)




## Forex Factory <a name="forex"></a>


Data from https://www.forexfactory.com/ was gotten using our own scrapper. Thus, we need to do some sanity checks to ensure that the downloaded data corresponds to the expected one.

As we have data from several years, the best approach for data curation is to create a script.
Before that, we need to explore the data for getting familiarity with our dataset. That´s exactly the goal of this notebook.


-----------


In [112]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz

In [113]:
# Global variables
# Please note this are relative directories to the project, so you need to edit these variables if modifying the folder structure

data_directory = '../../data/raw/'


### Initial exploration. Just for 2017, for the sake of getting familiarity with the data <a name="forex_explore"></a>


In [114]:
ff_2017 = pd.read_csv(data_directory + 'forexfactory_2017.csv')

In [115]:
ff_2017['datetime'] =  pd.to_datetime(ff_2017['datetime'])


In [116]:
ff_2017.head()

,Unnamed: 0,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week
0,0,NaN,NZD,2017-12-31,NaN,NaN,Non-Economic,Bank Holiday,NaN,NaN,52
1,1,NaN,AUD,2017-12-31,NaN,NaN,Non-Economic,Bank Holiday,NaN,NaN,52
2,2,NaN,JPY,2017-12-31,NaN,NaN,Non-Economic,Bank Holiday,NaN,NaN,52
3,3,NaN,CNY,2017-12-31,NaN,NaN,Non-Economic,Bank Holiday,NaN,NaN,52
4,4,NaN,NZD,2017-12-24,NaN,NaN,Non-Economic,Bank Holiday,NaN,NaN,52


In [117]:
ff_2017.dtypes

Unnamed: 0                 int64
actual                    object
country                   object
datetime          datetime64[ns]
forecast                  object
forecast_error            object
impact                    object
new                       object
previous                  object
previous_error            object
week                       int64
dtype: object

Let´s ensure the datetime column also has the time information

In [118]:
str(ff_2017['datetime'][16])

'2017-12-25 18:30:00'

In [119]:
ff_2017 = ff_2017.drop(columns = ['Unnamed: 0'])


Please note that **"forecast_error"** is a variable that I´ve created when scrapping the website, set to NaN whenever there was no error between the published forecast and the actual value.<br/> 
Equivalently, **"previous_error"** was also created by me, set to NaN whenever there was no goverment correction on the published value for the previous release event. 

Let´s replace those NaN by a categorical value = 'accurate'

In [120]:
ff_2017['forecast_error'] = ff_2017['forecast_error'].replace(np.nan, 'accurate', regex=True)
ff_2017['previous_error'] = ff_2017['previous_error'].replace(np.nan, 'accurate', regex=True)


Our preliminary analysis is going to be focused on **EUR-USD only**, analysing the impact of news published by the American government, so we filter the dataframe to only get **macroeconomic news from USA** (macroeconomic news = those which have a forecast)

In [121]:
ff_2017_USA = ff_2017[ff_2017['country'] == 'USD'] 
ff_2017_USA = ff_2017_USA[ff_2017_USA['forecast'].notnull()]
ff_2017_USA.head()

,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week
27,6.4%,USD,2017-12-26 09:00:00,6.3%,accurate,Low,S&P/CS Composite-20 HPI y/y,6.2%,accurate,52
28,20,USD,2017-12-26 09:59:00,22,worse,Low,Richmond Manufacturing Index,30,accurate,52
32,122.1,USD,2017-12-27 10:00:00,128.2,worse,High,CB Consumer Confidence,128.6,worse,52
33,0.2%,USD,2017-12-27 10:00:00,-0.4%,better,Medium,Pending Home Sales m/m,3.5%,accurate,52
41,245K,USD,2017-12-28 08:30:00,240K,worse,High,Unemployment Claims,245K,accurate,52


How many macro-economical news are published each year?

In [122]:
len(ff_2017_USA)

872

How many by 'impact' rate?

In [123]:
ff_2017_USA_high = ff_2017_USA[ff_2017_USA['impact'] == 'High']
ff_2017_USA_medium = ff_2017_USA[ff_2017_USA['impact'] == 'Medium']
ff_2017_USA_low = ff_2017_USA[ff_2017_USA['impact'] == 'Low']

print('High: ' + str(len(ff_2017_USA_high)) + ' - Medium: ' + str(len(ff_2017_USA_medium)) + ' - Low: ' + str(len(ff_2017_USA_low)))

High: 296 - Medium: 243 - Low: 333


Our favourite news for this analysis are those with high expected impact on the market. Let´s see how many of them we have

In [124]:
print('number of news, high: ' + 
      str(len(ff_2017_USA_high.groupby('new').impact.count())) +
      ' - med: ' +
        str(len(ff_2017_USA_medium.groupby('new').impact.count())) +
      ' - low: ' + 
        str(len(ff_2017_USA_low.groupby('new').impact.count())))

number of news, high: 22 - med: 27 - low: 29


Hmmm, not that many... :-(

Let´s see how many times forex factory publishes a wrong forecast

In [125]:
ff_2017_USA.groupby('forecast_error').impact.count()

forecast_error
accurate    301
better      288
worse       283
Name: impact, dtype: int64

Cool, forexfactory.com publishes non-accurate forecasts around 2/3 of the times !

Let´s see how often HIGH news are published

In [126]:
ff_2017_USA_high.groupby('new').impact.count()

new
ADP Non-Farm Employment Change    12
Advance GDP q/q                    4
Average Hourly Earnings m/m       12
Building Permits                  12
CB Consumer Confidence            12
CPI m/m                           12
Core CPI m/m                      12
Core Durable Goods Orders m/m     12
Core Retail Sales m/m             12
Crude Oil Inventories             52
Federal Funds Rate                 6
Final GDP q/q                      4
ISM Manufacturing PMI             12
ISM Non-Manufacturing PMI         12
Non-Farm Employment Change        12
PPI m/m                           12
Philly Fed Manufacturing Index     4
Prelim GDP q/q                     4
Prelim UoM Consumer Sentiment      4
Retail Sales m/m                  12
Unemployment Claims               50
Unemployment Rate                 12
Name: impact, dtype: int64

Mosts of them are monthly news. Let´s review one of them randomnly

In [127]:
ff_2017_USA[ff_2017_USA['new'] == 'ADP Non-Farm Employment Change']

,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week
278,190K,USD,2017-12-06 08:15:00,189K,accurate,High,ADP Non-Farm Employment Change,235K,accurate,49
764,235K,USD,2017-11-01 07:15:00,202K,better,High,ADP Non-Farm Employment Change,110K,worse,44
1111,135K,USD,2017-10-04 07:15:00,131K,accurate,High,ADP Non-Farm Employment Change,228K,worse,40
1547,237K,USD,2017-08-30 07:15:00,185K,better,High,ADP Non-Farm Employment Change,201K,better,35
1884,178K,USD,2017-08-02 07:15:00,187K,worse,High,ADP Non-Farm Employment Change,191K,better,31
2215,158K,USD,2017-07-06 07:15:00,184K,worse,High,ADP Non-Farm Employment Change,230K,worse,27
2672,253K,USD,2017-06-01 07:15:00,181K,better,High,ADP Non-Farm Employment Change,174K,accurate,22
2998,177K,USD,2017-05-03 07:15:00,178K,accurate,High,ADP Non-Farm Employment Change,255K,worse,18
3368,263K,USD,2017-04-05 07:15:00,184K,better,High,ADP Non-Farm Employment Change,245K,worse,14
3711,298K,USD,2017-03-08 08:15:00,184K,better,High,ADP Non-Farm Employment Change,261K,better,10


**Interesting...**
Forexfactory provided its data in US/Eastern with with DST = off (as I ran the scrapper during winter time). <br/>
This means that we need to manually add an extra hour whenever DST = on in US/Eastern. That´s exactly what forexfactory does.

Extra work to be done... After some time-consuming search on google, it´s easier than originaly thought.

In [128]:
def add_dts_flag(df):

    # Create a list of start and end dates for US in each year, in UTC time
    dst_changes_utc = pytz.timezone('US/Eastern')._utc_transition_times[1:]

    # Convert to local times from UTC times and then remove timezone information
    dst_changes = [pd.Timestamp(i).tz_localize('UTC').tz_convert('US/Eastern').tz_localize(None) for i in dst_changes_utc]

    flag_list = []
    for index, row in df['datetime'].iteritems():
        # Isolate the start and end dates for DST in each year
        dst_dates_in_year = [date for date in dst_changes if date.year == row.year]
        spring = dst_dates_in_year[0]
        fall = dst_dates_in_year[1]
        if (row >= spring) & (row < fall):
            flag = 1
        else:
            flag = 0
        flag_list.append(flag)
    
    return flag_list


In [129]:
ff_2017_USA['dst_flag'] = add_dts_flag(ff_2017_USA)
ff_2017_USA[ff_2017_USA['new'] == 'ADP Non-Farm Employment Change']

,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag
278,190K,USD,2017-12-06 08:15:00,189K,accurate,High,ADP Non-Farm Employment Change,235K,accurate,49,0
764,235K,USD,2017-11-01 07:15:00,202K,better,High,ADP Non-Farm Employment Change,110K,worse,44,1
1111,135K,USD,2017-10-04 07:15:00,131K,accurate,High,ADP Non-Farm Employment Change,228K,worse,40,1
1547,237K,USD,2017-08-30 07:15:00,185K,better,High,ADP Non-Farm Employment Change,201K,better,35,1
1884,178K,USD,2017-08-02 07:15:00,187K,worse,High,ADP Non-Farm Employment Change,191K,better,31,1
2215,158K,USD,2017-07-06 07:15:00,184K,worse,High,ADP Non-Farm Employment Change,230K,worse,27,1
2672,253K,USD,2017-06-01 07:15:00,181K,better,High,ADP Non-Farm Employment Change,174K,accurate,22,1
2998,177K,USD,2017-05-03 07:15:00,178K,accurate,High,ADP Non-Farm Employment Change,255K,worse,18,1
3368,263K,USD,2017-04-05 07:15:00,184K,better,High,ADP Non-Farm Employment Change,245K,worse,14,1
3711,298K,USD,2017-03-08 08:15:00,184K,better,High,ADP Non-Farm Employment Change,261K,better,10,0


Cool, it works pretty well. Let´s apply it to the dataframe.

In [130]:
def apply_dts_flag(row):
    return row['datetime'] + pd.DateOffset(hours=row['dst_flag'])

In [131]:
ff_2017_USA['datetime'] = ff_2017_USA.apply(apply_dts_flag, axis=1)


In [132]:
ff_2017_USA[ff_2017_USA['new'] == 'ADP Non-Farm Employment Change']

,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag
278,190K,USD,2017-12-06 08:15:00,189K,accurate,High,ADP Non-Farm Employment Change,235K,accurate,49,0
764,235K,USD,2017-11-01 08:15:00,202K,better,High,ADP Non-Farm Employment Change,110K,worse,44,1
1111,135K,USD,2017-10-04 08:15:00,131K,accurate,High,ADP Non-Farm Employment Change,228K,worse,40,1
1547,237K,USD,2017-08-30 08:15:00,185K,better,High,ADP Non-Farm Employment Change,201K,better,35,1
1884,178K,USD,2017-08-02 08:15:00,187K,worse,High,ADP Non-Farm Employment Change,191K,better,31,1
2215,158K,USD,2017-07-06 08:15:00,184K,worse,High,ADP Non-Farm Employment Change,230K,worse,27,1
2672,253K,USD,2017-06-01 08:15:00,181K,better,High,ADP Non-Farm Employment Change,174K,accurate,22,1
2998,177K,USD,2017-05-03 08:15:00,178K,accurate,High,ADP Non-Farm Employment Change,255K,worse,18,1
3368,263K,USD,2017-04-05 08:15:00,184K,better,High,ADP Non-Farm Employment Change,245K,worse,14,1
3711,298K,USD,2017-03-08 08:15:00,184K,better,High,ADP Non-Farm Employment Change,261K,better,10,0


As forexite was downloaded in UTC, no DTS, we would need to do the conversion before merging both dataframes

In [133]:
ff_2017_USA['datetime_utc'] = ff_2017_USA['datetime'] .dt.tz_localize('US/Eastern').dt.tz_convert('UTC')
ff_2017_USA[ff_2017_USA['new'] == 'ADP Non-Farm Employment Change']

,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag,datetime_utc
278,190K,USD,2017-12-06 08:15:00,189K,accurate,High,ADP Non-Farm Employment Change,235K,accurate,49,0,2017-12-06 13:15:00+00:00
764,235K,USD,2017-11-01 08:15:00,202K,better,High,ADP Non-Farm Employment Change,110K,worse,44,1,2017-11-01 12:15:00+00:00
1111,135K,USD,2017-10-04 08:15:00,131K,accurate,High,ADP Non-Farm Employment Change,228K,worse,40,1,2017-10-04 12:15:00+00:00
1547,237K,USD,2017-08-30 08:15:00,185K,better,High,ADP Non-Farm Employment Change,201K,better,35,1,2017-08-30 12:15:00+00:00
1884,178K,USD,2017-08-02 08:15:00,187K,worse,High,ADP Non-Farm Employment Change,191K,better,31,1,2017-08-02 12:15:00+00:00
2215,158K,USD,2017-07-06 08:15:00,184K,worse,High,ADP Non-Farm Employment Change,230K,worse,27,1,2017-07-06 12:15:00+00:00
2672,253K,USD,2017-06-01 08:15:00,181K,better,High,ADP Non-Farm Employment Change,174K,accurate,22,1,2017-06-01 12:15:00+00:00
2998,177K,USD,2017-05-03 08:15:00,178K,accurate,High,ADP Non-Farm Employment Change,255K,worse,18,1,2017-05-03 12:15:00+00:00
3368,263K,USD,2017-04-05 08:15:00,184K,better,High,ADP Non-Farm Employment Change,245K,worse,14,1,2017-04-05 12:15:00+00:00
3711,298K,USD,2017-03-08 08:15:00,184K,better,High,ADP Non-Farm Employment Change,261K,better,10,0,2017-03-08 13:15:00+00:00


-----------
On a different topic, we would also need to know which units are used per each macroeconomic new, so that we can compute the error rate between forecast and reality.

In [134]:
list(set(ff_2017_USA.groupby('new').first().forecast))

['58.4',
 '97.1',
 '-135.2B',
 '-67.7B',
 '104.6',
 '99.0',
 '198K',
 '62.2',
 '0.7%',
 '57.6B',
 '-116B',
 '0.5%',
 '2.1%',
 '3.3%',
 '-3.9M',
 '-0.4%',
 '18.8',
 '54.8',
 '22',
 '1.27M',
 '4.1%',
 '2.6%',
 '0.6%',
 '0.4%',
 '17.5M',
 '6.3%',
 '2.5%',
 '6.03M',
 '54.0',
 '-0.1%',
 '-115B',
 '55.4',
 '5.53M',
 '1.25M',
 '-0.3%',
 '189K',
 '2.2%',
 '59.2',
 '240K',
 '654K',
 '1.7%',
 '17.4B',
 '0.2%',
 '-46.2B',
 '0.3%',
 '70',
 '77.2%',
 '54.6',
 '67.0',
 '21.5',
 '<1.50%',
 '53.8',
 '0.1%',
 '128.2']

OK, again this is not nice... Extra processing will need to be done to compute error_ratio

--- 

## Next steps on Forex Factory <a name="next_forex"></a>


### Sanity checks:

 - No missing weeks. Each year should have 52 weeks.
 
### Data selection:

 - Filter just macro-economic news.
 - Filter news just on the currency of interest.
 
### Feature Engineer:

 - Replace NaN in "forecast_error" and "previous_error" fields by "accurate".
 - Manually add +1h to forexfactory data to account for DTS (date time savings).
 - Set all timestamps to match the trading pair values got from Forexite, i.e. GMT without DTS. Otherwise we won´t compare apples with apples !
 - Compute percentage of error between the forecasted and actual values, taking into account the different units handled (int, float, %, Millions = 'M', Thousands = 'K').
 - Add year, quarter, month, day of week as caegorical variables.


<br/>

----

## Forexite <a name="forexite"></a>


### Initial exploration. Just EUR-USD, for the sake of getting familiarity with the data <a name="forexite_explore"></a>




Currency data from https://forextester.com/data/datasources is already provided as csv files, one per each currency pair.



In [135]:
eurusd = pd.read_csv(data_directory + 'EURUSD.txt.zip', compression='zip', header=0, sep=',')


In [136]:
eurusd.head()

,<TICKER>,<DTYYYYMMDD>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,EURUSD,20010102,230100,0.9507,0.9507,0.9507,0.9507,4
1,EURUSD,20010102,230200,0.9506,0.9506,0.9505,0.9505,4
2,EURUSD,20010102,230300,0.9505,0.9507,0.9505,0.9506,4
3,EURUSD,20010102,230400,0.9506,0.9506,0.9506,0.9506,4
4,EURUSD,20010102,230500,0.9506,0.9506,0.9506,0.9506,4


In [137]:
eurusd.dtypes

<TICKER>         object
<DTYYYYMMDD>      int64
<TIME>            int64
<OPEN>          float64
<HIGH>          float64
<LOW>           float64
<CLOSE>         float64
<VOL>             int64
dtype: object

The data is listed by minute. We won´t be interested is such degree of granularity. For our study, we will need to group this data into broader chunks

In [138]:
eurusd.describe()

,<DTYYYYMMDD>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
count,6.239078e+06,6.239078e+06,6.239078e+06,6.239078e+06,6.239078e+06,6.239078e+06,6239078.0
mean,2.009711e+07,1.163738e+05,1.239739e+00,1.239813e+00,1.239666e+00,1.239740e+00,4.0
std,5.018670e+04,6.811457e+04,1.555687e-01,1.555863e-01,1.555514e-01,1.555688e-01,0.0
min,2.001010e+07,0.000000e+00,8.350000e-01,8.350000e-01,8.350000e-01,8.350000e-01,4.0
25%,2.005121e+07,6.010000e+04,1.136400e+00,1.136500e+00,1.136300e+00,1.136400e+00,4.0
50%,2.010031e+07,1.149000e+05,1.261900e+00,1.262000e+00,1.261800e+00,1.261900e+00,4.0
75%,2.014061e+07,1.735000e+05,1.346400e+00,1.346400e+00,1.346300e+00,1.346400e+00,4.0
max,2.018093e+07,2.359000e+05,1.603300e+00,1.603600e+00,1.602700e+00,1.603400e+00,4.0


--- 

## Feature engineer using Forexite data <a name="next_forexite"></a>

Forexite provides the exchange rate for the major pairs of interest. We will use this data to evaluate the impact in that pair created by the releases of macroeconomic data

#### Situation of the market _before_ publishing the new:
    
 - Create a new dataframe, grouping the data per day (open, high, low, close).
 - Add 12 new features to the news dataframe -> (open, high, low, close) for the 3 days prior to the new publication.

#### Situation of the market _after_ publishing the new:

 - 5/10/15/39/45/60/90-min window size (volatility (high - low), direction (up|down), close).


## Feature engineer <a name="feature_engineer"></a>

 - Dataframe:
      - surprise_forecast
      - surprise_volatility
     

<br/>

----

In [139]:
eurusd = pd.read_csv(data_directory + 'EURUSD.txt.zip', compression='zip', header=0, sep=',', \
                     dtype={'<DTYYYYMMDD>': 'str', '<TIME>': 'str'})

eurusd.head(2)

,<TICKER>,<DTYYYYMMDD>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,EURUSD,20010102,230100,0.9507,0.9507,0.9507,0.9507,4
1,EURUSD,20010102,230200,0.9506,0.9506,0.9505,0.9505,4


In [140]:
eurusd['datetime'] = eurusd['<DTYYYYMMDD>'] + eurusd['<TIME>']

In [141]:
eurusd.head(2)

,<TICKER>,<DTYYYYMMDD>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,datetime
0,EURUSD,20010102,230100,0.9507,0.9507,0.9507,0.9507,4,20010102230100
1,EURUSD,20010102,230200,0.9506,0.9506,0.9505,0.9505,4,20010102230200


In [142]:
eurusd['datetime_utc'] = pd.to_datetime(eurusd['datetime'], format='%Y%m%d%H%M%S', errors='raise', utc =True)

In [143]:
eurusd.head(2)

,<TICKER>,<DTYYYYMMDD>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,datetime,datetime_utc
0,EURUSD,20010102,230100,0.9507,0.9507,0.9507,0.9507,4,20010102230100,2001-01-02 23:01:00+00:00
1,EURUSD,20010102,230200,0.9506,0.9506,0.9505,0.9505,4,20010102230200,2001-01-02 23:02:00+00:00


In [144]:
eurusd.dtypes

<TICKER>                     object
<DTYYYYMMDD>                 object
<TIME>                       object
<OPEN>                      float64
<HIGH>                      float64
<LOW>                       float64
<CLOSE>                     float64
<VOL>                         int64
datetime                     object
datetime_utc    datetime64[ns, UTC]
dtype: object

In [145]:
eurusd = eurusd.set_index('datetime_utc')
eurusd.head(2)

,<TICKER>,<DTYYYYMMDD>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,datetime
datetime_utc,,,,,,,,,
2001-01-02 23:01:00+00:00,EURUSD,20010102,230100,0.9507,0.9507,0.9507,0.9507,4,20010102230100
2001-01-02 23:02:00+00:00,EURUSD,20010102,230200,0.9506,0.9506,0.9505,0.9505,4,20010102230200


In [146]:
eurusd[eurusd.index == '2017-03-12 23:01:00+00:00']

,<TICKER>,<DTYYYYMMDD>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,datetime
datetime_utc,,,,,,,,,
2017-03-12 23:01:00+00:00,EURUSD,20170312,230100,1.0677,1.0678,1.0677,1.0677,4,20170312230100


In [147]:
eurusd = eurusd.drop(['<DTYYYYMMDD>','<TIME>','<VOL>', 'datetime'], axis=1)
eurusd.head(2)

,<TICKER>,<OPEN>,<HIGH>,<LOW>,<CLOSE>
datetime_utc,,,,,
2001-01-02 23:01:00+00:00,EURUSD,0.9507,0.9507,0.9507,0.9507
2001-01-02 23:02:00+00:00,EURUSD,0.9506,0.9506,0.9505,0.9505


In [148]:
eurusd.columns = ['pair', 'open', 'high', 'low', 'close']
eurusd.head(2)

,pair,open,high,low,close
datetime_utc,,,,,
2001-01-02 23:01:00+00:00,EURUSD,0.9507,0.9507,0.9507,0.9507
2001-01-02 23:02:00+00:00,EURUSD,0.9506,0.9506,0.9505,0.9505


In [149]:
eurusd_5m_agg = eurusd.groupby(pd.Grouper(freq='5Min')).agg(
            {'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'})

In [150]:
eurusd_5m_agg[eurusd_5m_agg.index == '2001-01-03 02:05:00']

,open,high,low,close
datetime_utc,,,,
2001-01-03 02:05:00+00:00,0.95,0.9501,0.9498,0.9498


In [151]:
eurusd_5m_agg.dtypes

open     float64
high     float64
low      float64
close    float64
dtype: object

In [152]:
eurusd_5m_agg.head()

,open,high,low,close
datetime_utc,,,,
2001-01-02 23:00:00+00:00,0.9507,0.9507,0.9505,0.9506
2001-01-02 23:05:00+00:00,0.9506,0.9507,0.9505,0.9507
2001-01-02 23:10:00+00:00,0.9507,0.9507,0.9506,0.9507
2001-01-02 23:15:00+00:00,0.9507,0.9507,0.9506,0.9506
2001-01-02 23:20:00+00:00,0.9507,0.9507,0.9507,0.9507


In [153]:
eurusd_5m_agg.index

DatetimeIndex(['2001-01-02 23:00:00+00:00', '2001-01-02 23:05:00+00:00',
               '2001-01-02 23:10:00+00:00', '2001-01-02 23:15:00+00:00',
               '2001-01-02 23:20:00+00:00', '2001-01-02 23:25:00+00:00',
               '2001-01-02 23:30:00+00:00', '2001-01-02 23:35:00+00:00',
               '2001-01-02 23:40:00+00:00', '2001-01-02 23:45:00+00:00',
               ...
               '2018-09-30 23:10:00+00:00', '2018-09-30 23:15:00+00:00',
               '2018-09-30 23:20:00+00:00', '2018-09-30 23:25:00+00:00',
               '2018-09-30 23:30:00+00:00', '2018-09-30 23:35:00+00:00',
               '2018-09-30 23:40:00+00:00', '2018-09-30 23:45:00+00:00',
               '2018-09-30 23:50:00+00:00', '2018-09-30 23:55:00+00:00'],
              dtype='datetime64[ns, UTC]', name='datetime_utc', length=1866252, freq='5T')

In [154]:
ff_2017_USA.head(2)

,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag,datetime_utc
27,6.4%,USD,2017-12-26 09:00:00,6.3%,accurate,Low,S&P/CS Composite-20 HPI y/y,6.2%,accurate,52,0,2017-12-26 14:00:00+00:00
28,20,USD,2017-12-26 09:59:00,22,worse,Low,Richmond Manufacturing Index,30,accurate,52,0,2017-12-26 14:59:00+00:00


In [155]:
ff_2017_USA['a_5_min'] = ff_2017_USA['datetime_utc'] + pd.DateOffset(minutes=5)


In [156]:
ff_2017_USA.head(2)

,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag,datetime_utc,a_5_min
27,6.4%,USD,2017-12-26 09:00:00,6.3%,accurate,Low,S&P/CS Composite-20 HPI y/y,6.2%,accurate,52,0,2017-12-26 14:00:00+00:00,2017-12-26 14:05:00+00:00
28,20,USD,2017-12-26 09:59:00,22,worse,Low,Richmond Manufacturing Index,30,accurate,52,0,2017-12-26 14:59:00+00:00,2017-12-26 15:04:00+00:00


In [157]:
ff_2017_USA.set_index('a_5_min').head(2)

,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag,datetime_utc
a_5_min,,,,,,,,,,,,
2017-12-26 14:05:00+00:00,6.4%,USD,2017-12-26 09:00:00,6.3%,accurate,Low,S&P/CS Composite-20 HPI y/y,6.2%,accurate,52,0,2017-12-26 14:00:00+00:00
2017-12-26 15:04:00+00:00,20,USD,2017-12-26 09:59:00,22,worse,Low,Richmond Manufacturing Index,30,accurate,52,0,2017-12-26 14:59:00+00:00


In [158]:
#ff_2017_USA['a_5_min'] = ff_2017_USA['a_5_min'].dt.tz_localize(None)

In [159]:
len(ff_2017_USA)

872

In [160]:
df = ff_2017_USA.set_index('a_5_min').join(eurusd_5m_agg)

In [161]:
len(df)

872

In [162]:
df.head(2)

,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag,datetime_utc,open,high,low,close
2017-01-03 14:50:00+00:00,54.3,USD,2017-01-03 09:45:00,54.2,accurate,Low,Final Manufacturing PMI,54.2,accurate,1,0,2017-01-03 14:45:00+00:00,1.0383,1.0387,1.0383,1.0386
2017-01-03 15:05:00+00:00,54.7,USD,2017-01-03 10:00:00,53.7,better,High,ISM Manufacturing PMI,53.2,accurate,1,0,2017-01-03 15:00:00+00:00,1.0388,1.0388,1.0385,1.0386


In [163]:
df[df.index == '2017-12-26 14:05:00']

,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag,datetime_utc,open,high,low,close
2017-12-26 14:05:00+00:00,6.4%,USD,2017-12-26 09:00:00,6.3%,accurate,Low,S&P/CS Composite-20 HPI y/y,6.2%,accurate,52,0,2017-12-26 14:00:00+00:00,1.1848,1.1849,1.1848,1.1849


In [164]:
df = df.reset_index()
df.head(2)

,index,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag,datetime_utc,open,high,low,close
0,2017-01-03 14:50:00+00:00,54.3,USD,2017-01-03 09:45:00,54.2,accurate,Low,Final Manufacturing PMI,54.2,accurate,1,0,2017-01-03 14:45:00+00:00,1.0383,1.0387,1.0383,1.0386
1,2017-01-03 15:05:00+00:00,54.7,USD,2017-01-03 10:00:00,53.7,better,High,ISM Manufacturing PMI,53.2,accurate,1,0,2017-01-03 15:00:00+00:00,1.0388,1.0388,1.0385,1.0386


In [165]:
len(df[df.isnull().any(1)])

23

In [166]:
len(df)

872

In [167]:
df1 = df.dropna()

In [168]:
len(df1)

849

In [169]:
23*100/872

2.6376146788990824

In [170]:
df.to_csv('test.csv')

In [171]:
df.head(2)

,index,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag,datetime_utc,open,high,low,close
0,2017-01-03 14:50:00+00:00,54.3,USD,2017-01-03 09:45:00,54.2,accurate,Low,Final Manufacturing PMI,54.2,accurate,1,0,2017-01-03 14:45:00+00:00,1.0383,1.0387,1.0383,1.0386
1,2017-01-03 15:05:00+00:00,54.7,USD,2017-01-03 10:00:00,53.7,better,High,ISM Manufacturing PMI,53.2,accurate,1,0,2017-01-03 15:00:00+00:00,1.0388,1.0388,1.0385,1.0386


In [172]:
df.rename({'actual':'actualpp'}, axis='columns').head(2)

,index,actualpp,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag,datetime_utc,open,high,low,close
0,2017-01-03 14:50:00+00:00,54.3,USD,2017-01-03 09:45:00,54.2,accurate,Low,Final Manufacturing PMI,54.2,accurate,1,0,2017-01-03 14:45:00+00:00,1.0383,1.0387,1.0383,1.0386
1,2017-01-03 15:05:00+00:00,54.7,USD,2017-01-03 10:00:00,53.7,better,High,ISM Manufacturing PMI,53.2,accurate,1,0,2017-01-03 15:00:00+00:00,1.0388,1.0388,1.0385,1.0386


In [173]:
df.head(2)

,index,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag,datetime_utc,open,high,low,close
0,2017-01-03 14:50:00+00:00,54.3,USD,2017-01-03 09:45:00,54.2,accurate,Low,Final Manufacturing PMI,54.2,accurate,1,0,2017-01-03 14:45:00+00:00,1.0383,1.0387,1.0383,1.0386
1,2017-01-03 15:05:00+00:00,54.7,USD,2017-01-03 10:00:00,53.7,better,High,ISM Manufacturing PMI,53.2,accurate,1,0,2017-01-03 15:00:00+00:00,1.0388,1.0388,1.0385,1.0386


-----
debug

In [174]:
ff_2017_USA[ff_2017_USA['new'] == 'Unemployment Rate']

,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,week,dst_flag,datetime_utc,a_5_min
332,4.1%,USD,2017-12-08 08:30:00,4.1%,accurate,High,Unemployment Rate,4.1%,accurate,49,0,2017-12-08 13:30:00+00:00,2017-12-08 13:35:00+00:00
816,4.1%,USD,2017-11-03 08:30:00,4.2%,better,High,Unemployment Rate,4.2%,accurate,44,1,2017-11-03 12:30:00+00:00,2017-11-03 12:35:00+00:00
1152,4.2%,USD,2017-10-06 08:30:00,4.4%,better,High,Unemployment Rate,4.4%,accurate,40,1,2017-10-06 12:30:00+00:00,2017-10-06 12:35:00+00:00
1596,4.4%,USD,2017-09-01 08:30:00,4.3%,worse,High,Unemployment Rate,4.3%,accurate,35,1,2017-09-01 12:30:00+00:00,2017-09-01 12:35:00+00:00
1924,4.3%,USD,2017-08-04 08:30:00,4.3%,accurate,High,Unemployment Rate,4.4%,accurate,31,1,2017-08-04 12:30:00+00:00,2017-08-04 12:35:00+00:00
2247,4.4%,USD,2017-07-07 08:30:00,4.3%,worse,High,Unemployment Rate,4.3%,accurate,27,1,2017-07-07 12:30:00+00:00,2017-07-07 12:35:00+00:00
2694,4.3%,USD,2017-06-02 08:30:00,4.4%,better,High,Unemployment Rate,4.4%,accurate,22,1,2017-06-02 12:30:00+00:00,2017-06-02 12:35:00+00:00
3044,4.4%,USD,2017-05-05 08:30:00,4.6%,better,High,Unemployment Rate,4.5%,accurate,18,1,2017-05-05 12:30:00+00:00,2017-05-05 12:35:00+00:00
3412,4.5%,USD,2017-04-07 08:30:00,4.7%,better,High,Unemployment Rate,4.7%,accurate,14,1,2017-04-07 12:30:00+00:00,2017-04-07 12:35:00+00:00
3756,4.7%,USD,2017-03-10 08:30:00,4.7%,accurate,High,Unemployment Rate,4.8%,accurate,10,0,2017-03-10 13:30:00+00:00,2017-03-10 13:35:00+00:00


In [176]:
df = pd.read_csv('../../data/curated/macroeconomic_news_2007_2018.csv')
df.head(2)

,Unnamed: 0,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,...,volatility_60,direction_60,open_90,close_90,volatility_90,direction_90,open_120,close_120,volatility_120,direction_120
0,0,12.3B,USD,2007-01-08 15:00:00,5.4B,better,Low,Consumer Credit m/m,-1.3B,accurate,...,1.0,down,1.3018,1.3019,1.0,up,1.3021,1.3021,2.0,down
1,1,53.7,USD,2007-01-09 10:00:00,53.7,accurate,Low,IBD/TIPP Economic Optimism,53.5,accurate,...,5.0,up,1.2995,1.2997,2.0,up,1.2993,1.2994,3.0,up


In [192]:
df[(df['new'] == 'Unemployment Rate') & (df['year'] == 2013)]

,Unnamed: 0,actual,country,datetime,forecast,forecast_error,impact,new,previous,previous_error,...,volatility_60,direction_60,open_90,close_90,volatility_90,direction_90,open_120,close_120,volatility_120,direction_120
4565,8,7.8%,USD,2013-01-04 08:30:00,7.7%,worse,High,Unemployment Rate,7.7%,accurate,...,10.0,down,1.3055,1.3047,15.0,down,1.3044,1.3040,8.0,down
4628,72,7.9%,USD,2013-02-01 08:30:00,7.8%,worse,High,Unemployment Rate,7.8%,accurate,...,13.0,down,1.3591,1.3595,11.0,up,1.3637,1.3641,9.0,up
4708,154,7.7%,USD,2013-03-08 08:30:00,7.9%,better,High,Unemployment Rate,7.9%,accurate,...,8.0,up,1.2976,1.2988,19.0,up,1.2988,1.2985,7.0,down
4771,220,7.6%,USD,2013-04-05 08:30:00,7.7%,better,High,Unemployment Rate,7.7%,accurate,...,6.0,up,1.2999,1.3005,15.0,up,1.3002,1.3000,4.0,down
4836,286,7.5%,USD,2013-05-03 08:30:00,7.6%,better,High,Unemployment Rate,7.6%,accurate,...,21.0,up,1.3112,1.3119,21.0,up,1.3141,1.3141,15.0,down
4908,359,7.6%,USD,2013-06-07 08:30:00,7.5%,worse,High,Unemployment Rate,7.5%,accurate,...,9.0,down,1.3202,1.3200,13.0,down,1.3213,1.3210,7.0,down
4972,426,7.6%,USD,2013-07-05 08:30:00,7.5%,worse,High,Unemployment Rate,7.6%,accurate,...,6.0,up,1.2824,1.2832,15.0,up,1.2831,1.2837,9.0,up
5027,486,7.4%,USD,2013-08-02 08:30:00,7.5%,better,High,Unemployment Rate,7.6%,accurate,...,6.0,down,1.3263,1.3263,12.0,down,1.3283,1.3279,7.0,down
5104,565,7.3%,USD,2013-09-06 08:30:00,7.4%,better,High,Unemployment Rate,7.4%,accurate,...,8.0,down,1.3165,1.3183,21.0,up,1.3160,1.3156,8.0,down
5178,641,7.2%,USD,2013-10-22 08:30:00,7.3%,better,High,Unemployment Rate,7.3%,accurate,...,9.0,up,1.3751,1.3761,12.0,up,1.3770,1.3766,6.0,down
